In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm #Makes iterations look better
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
train_data = pd.read_csv('../input/train.csv',dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})


In [ ]:
# Display the head of the dataframe
pd.options.display.precision = 10

train_data.head()

In [ ]:
# Dimensions of the given training data
print("Rows: {}, Columns: {}".format(train_data.shape[0],train_data.shape[1]))

In [ ]:
# Create segments of 150000 data points as mentioned by the competition organisers.
segment_size = 150000
num_segments = int(np.floor(train_data.shape[0]/segment_size))

X_train = pd.DataFrame(index=range(num_segments),columns=['ave','std','min','max'],dtype=np.float64)
y_train = pd.DataFrame(index=range(num_segments),columns=['time_to_failure'],dtype=np.float64)

for i in tqdm(range(num_segments)):
    segment_i = train_data.iloc[i*segment_size:i*segment_size+segment_size]
    x = segment_i['acoustic_data'].values
    y = segment_i['time_to_failure'].values[-1]
    X_train.loc[i,'ave'] = x.mean()
    X_train.loc[i,'std'] = x.std()
    X_train.loc[i,'max'] = x.max()
    X_train.loc[i,'min'] = x.min()
    
    y_train.loc[i,'time_to_failure'] = y
    

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
print("X_train Shape: {}, y_train Shape: {}".format(X_train.shape,y_train.shape))

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled

In [40]:
# Create testing data / handle the testing part

submission_files = pd.read_csv('../input/sample_submission.csv',index_col='seg_id')
submission_files

X_test = pd.DataFrame(columns=X_train.columns,index=submission_files.index,dtype=np.float64)

for seg_id in tqdm(X_test.index):
    segment = pd.read_csv('../input/test/'+seg_id+'.csv')
    x = segment['acoustic_data'].values
    X_test.loc[seg_id,'ave'] = x.mean()
    X_test.loc[seg_id,'std'] = x.std()
    X_test.loc[seg_id,'max'] = x.max()
    X_test.loc[seg_id,'min'] = x.min()
X_test

100%|██████████| 2624/2624 [00:38<00:00, 71.77it/s]


,ave,std,min,max
seg_id,,,,
seg_00030f,4.4917800000,4.8936733747,-75.0,115.0
seg_0012b5,4.1711533333,5.9228197004,-140.0,152.0
seg_00184e,4.6102600000,6.9469669208,-193.0,248.0
seg_003339,4.5314733333,4.1141328891,-93.0,85.0
seg_0042cc,4.1283400000,5.7971443123,-147.0,177.0
seg_004314,4.1486066667,24.7826862156,-675.0,671.0
seg_004cd2,4.1139866667,4.7071343412,-107.0,125.0
seg_004ee5,4.3283800000,5.9644234068,-120.0,120.0
seg_004f1f,4.0007333333,5.8744497157,-114.0,118.0


In [42]:
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[-0.1081212 , -0.19460464,  0.27998382, -0.17788929],
       [-1.36033181, -0.07358216,  0.03477659, -0.04232056],
       [ 0.35460361,  0.04685245, -0.16516161,  0.30942533],
       ...,
       [-3.01067009,  0.34325463, -0.58012768,  0.53659455],
       [-0.69431219,  0.78209681, -0.64048638,  0.72712357],
       [-1.24876441, -0.34988847,  0.34788736, -0.33177812]])

In [43]:
model = GradientBoostingRegressor(learning_rate=0.1,n_estimators=200,loss='ls').fit(X_train_scaled,y_train.values.flatten())
y_predictions = model.predict(X_train_scaled)
y_predictions

array([ 5.69300073,  4.19321843,  3.63405259, ..., 10.83218703,
        8.40500947,  9.11570207])

In [44]:
# Mean absolute error on training data itself

mean_absolute_error(y_train.values.flatten(),y_predictions)

2.009756441255235

In [47]:
y_test_predictions = model.predict(X_test_scaled)
submission_files['time_to_failure'] = y_test_predictions
submission_files.to_csv('submission1.csv')